In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron


iris= load_iris()
x = iris.data[:, (2,3)] # petal length, petal width
y = (iris.target == 0).astype(np.int) # iris setosa

In [6]:
per_clf = Perceptron(random_state = 42)
per_clf.fit(x, y)
y_pred = per_clf.predict([[2,0.5], [1,2], [3,4]])

In [7]:
print(y_pred)

[1 0 0]


In [8]:
# using the stochastic gradient descent algorithm
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss = 'perceptron', learning_rate = 'constant', eta0 = 1, penalty = None)
sgd.fit(x, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=1, fit_intercept=True, l1_ratio=0.15, learning_rate='constant',
       loss='perceptron', n_iter=5, n_jobs=1, penalty=None, power_t=0.5,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [9]:
y_pred = sgd.predict([[2,0.5],[1,2],[3,4]])
print(y_pred)

[0 0 0]


In [ ]:
# training an MLP with TensorFlow's high-level API on the Mnist image dataset

import tensorflow as tf
feature_cols= tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units = [300,100], n_classes = 10, 
                                        feature_columns = feature_cols)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if Tensorflow >= 1.1
dnn_clf.fit(X_train, y_train, batch_size = 50, steps = 40000)


from sklearn.metrics import  accuracy_score
y_pred = dnn_clf.predict(x_test)
accuracy_score(y_test, y_pred['classes'])

In [11]:
import tensorflow as tf
help (tf.truncated_normal)

Help on function truncated_normal in module tensorflow.python.ops.random_ops:

truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
    Outputs random values from a truncated normal distribution.
    
    The generated values follow a normal distribution with specified mean and
    standard deviation, except that values whose magnitude is more than 2 standard
    deviations from the mean are dropped and re-picked.
    
    Args:
      shape: A 1-D integer Tensor or Python array. The shape of the output tensor.
      mean: A 0-D Tensor or Python value of type `dtype`. The mean of the
        truncated normal distribution.
      stddev: A 0-D Tensor or Python value of type `dtype`. The standard deviation
        of the truncated normal distribution.
      dtype: The type of the output.
      seed: A Python integer. Used to create a random seed for the distribution.
        See
        @{tf.set_random_seed}
        for behavior.
      name: A name for the 

In [ ]:
# training a DNN using plain tensorflow
n_inputs = 28 *28 # MNIST no features is 28 by 28(one feature per pixel)
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# since we do not know how many instances each training batch will contain, so the shape
# of X is (None, n_inputs) and y is (None)

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'X')
y = tf.placeholder(tf.int64, shape = (None), name= 'y')

# creating the two hidden layers and the output, the output layer uses a softmax activation
# function instead of a RELU activation function.
# we will create a nuuron_layer() function that we will use to create one layer at a time
# It will need parameters to specify the inputs, the number of neutrons, the activation function,
# and the name of the layer....

def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.truncated_normal((n_inputs, n_neurons),stddev = stddev)
        W = tf.Variable(init,name = 'kernel')
        b = tf.Variable(tf.zeros(n_neurons), name = 'bias')
        z = tf.matmul(X, W) + b
        if activation is not None: # that is if activation parameter is provided
            # such as tf.nn.relu(i.e, max(0,z)), then the code returns activation(Z), or
            #else it just returns Z.
            return activation(z)
        else:
            return z
        
        
        
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, n_hidden1, name = 'hidden1', activation = tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name = 'hidden2', activation = tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name = 'outputs')
    
    
# using tensorflow built in support for building layers
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, name = 'hidden1', activation = tf.nn.relu)
    hidden2 = tf.layers.dense(x, n_hidden2, name = 'hidden2', activation = tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name = 'outputs')
    
    
# applyiing cross Entropy, cross entropy will penalize models that estimates a low
# probability for the target class. using sparse_softmax_cross_entropy_with_logits()
# it computes the cross entropy based on the 'logits' (i.e., the output of the network)
# before going through the softmax activation function), and it expects labels in the form
# of integers ranging from 0 to the number of classes minus 1(in our case, from 0 to 9). This
# will give us a 1D tensor containing the cross entropy for each instance. 
# we can then use Tensorflow's reduce_mean() function to compute the mean cross entropy over all
# instances.

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')


In [ ]:
# now getting the GradientDescentOptimizer that will tweak the model parameters, to
# minimize the cost function

learning_rate = 0.01

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    trainin_op = optimizer.minimize(loss)
    
    
# now to specify how to evaluate our model, simply using accuracy as our performance measure
# to determine if the neural network's prediction is correct by checking whether or not
# the highest logit corresponds to the target class... using the in_to_k() function.
# this returns a 1D tensor full of boolean values, so we need to cast these booleans to floats
# and the compute the average.

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y,1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

# Execution phase

In [ ]:
from tensorflow.examples.tutorial.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

n_epochs = 40
batch_size = 50

with tf.Session() as ess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
            acc_train = accuracy.eval(feed_dict = {X:X_batch, y:y_batch})
            acc_val = accuracy.eval(feed_dict = {X:mnist.validation.images, y:mnist.validation.labels})
            print(epoch, 'Train accuracy:', acc_train, 'Val accuracy:', acc_val)
            
        save_path = saver.save(sess, './my_model_final.ckpt')

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, './my_model_final.ckpt')
    X_new_scaled = [....] # some new images(scaled from 0 to 1)
    z = logits.eval(feed_dict = {X:X_new_scaled})
    y_pred = np.argmax(z, axis = 1)

In [ ]:
# fine-tuned neural network hyperparameters



# for the output layer, the softmax activation function is generally a good choice
# for classification tasks when the classes are mutually exclusive. When they are not
# mutally exclusive(or when they are just two classes), you generally want to use the 
# logistic function. For regression tasks, you can simply use no activation function at all
# for the output layer.